## Climate Change and the Optimal Decisions of Vineyard Farmers

#### Background

Wine industry is highly sensitive to the nuances of climate and terroir, is facing unprecedented challenges due to climate change. And climate change will affect vineyards production through shifts in temperature and precipitation patterns. 

One response to the climatic shift is "migration" -- relocate to a more climatically suitable area. But this strategy might incurr a potential loss of location-specific premium associated with established wine-producing regions.

Another possible stretagy is "adaptation". This includes introducing new grape varieties, using new techniques, rethinking vineyard orientations, etc. And this will increase the cost of production. 

#### Problem Statement

We aim to employ a Reinforcement Learning (RL) model to optimize strategies in the context of climate change. The model is designed to operate under varying temperature scenarios, with each scenario influencing the vineyard's state and the effectiveness of potential adaptation actions.  

The RL model framework involves:

__1.Space State ($S$):__
Defined by a range of temperature scenarios and other relevant vineyard conditions. Each state $s\in S$ represents specific environmental and climatic conditions.

__2.Action Space($A$):__

__3.Object Function:__


\begin{eqnarray}
\text{minimize}~\mathcal{C}_T &=& \sum{i\in{1,\dotsc,n}}c_{i}x_{i} \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}p_{i}x_{i} & \leq & P_{\text{max}}\\
\text{and}~\mathbf{C}\mathbf{x} & \leq & \mathbf{b} \\
\text{and}~x_{i}&\in&{0,1}\qquad{i=1,2,\dots,n}
\end{eqnarray}

In this model:

* $c_{i}\geq 0$ denotes the cost of adopting strategy$i$. 
* $p_{i}$ denotes the potential impact on the location premium by adopting strategy$i$, with $P_{max}$ representing the maximum permissible impact on the premium.
* $x_{i}\in{0,1}$ represents the binary decision of adopting or not adopting strategy$i$. 


#### List of Tasks
* __Task 1__: Specify the vectors
* __Task 2__: 

### Setup

In [1]:
include("CHEME-5760-L13a-CodeLib.jl");

In [2]:
using Plots
using Colors

## Task 1: Specify the Vectors

In this problem, we will choose Pinot Grins as baseline.

The optimal temperature for Pinot Grins to grow is between 13-15 celsius degree (55.4 - 59 Fahrenheit). 

There are several strategies can be taken

### Adaptation Strategies

__1.__ Adjust harvest dates

* Estimated Cost: $0 per acre
* Temperature Offset: 1 per 14 days - but date adjustment cannot be larger than 30 days

__2.__ Switch to another cultivar (assume from Pinot Grins to Cabernet Sauvignon)

* Estimated Cost: $824.6 per acre (the yield per harvested acre for Pinot Grins is 4.74, and price pre ton is 1800 -> 8532 per acre; the yield per harvested acre for Pinot Grins is 3.01, and price pre ton is 2560 -> 7705.6 per acre)
* Temperature Offset: 9 the optimal temperature for Cabernet Sauvignon is 16.5-20 (61.7-68)

__3.__ Production technology -- full-capacity watering

* Estimated Cost: 500-1200 USD per acre
* Temperature Offset: 2-4

__4.__ Production technology -- canopy manipulation

* Estimated Cost: 300-800 USD per acre
* Temperature Offset: 1-3

__5.__ Production technology -- changing row orientation

* Estimated Cost: 5000-10000 USD per acre (one-time cost)
* Temperature Offset: 2-4


### Migration

__6.__ Northward movement

* Estimated Cost (premium loss): $6930 per acre
* Temperature Offset: will reach the optimal temperature

In [3]:
# build a model of the states (temperature)
number_of_rows = 121 # temperature range from 0 - 120 fahrenheit
number_of_cols = 121

nstates = (number_of_rows*number_of_cols);
min_temp = 0
max_temp = 120
min_temp_range = 0:1:120
max_temp_range = 0:1:120 
𝒮 = [(min_temp, max_temp) for min_temp in min_temp_range for max_temp in max_temp_range if min_temp <= max_temp]

nactions = 6
𝒜 = 1:nactions
action_mapping = Dict(1 => "HarvestDate", 2 => "ChangeVineType", 3 => "FullCapacityWatering", 
                      4 => "CanopyManipulation", 5 => "ChangeRowOrientation", 6 => "NorthwardMovement")

Dict{Int64, String} with 6 entries:
  5 => "ChangeRowOrientation"
  4 => "CanopyManipulation"
  6 => "NorthwardMovement"
  2 => "ChangeVineType"
  3 => "FullCapacityWatering"
  1 => "HarvestDate"

In [4]:
# define costs and temperature offsets
action_costs = [0.00, 824.60, 750.00, 550.00, 7500.00, 6930.00] 
temperature_offsets = [0.07, 9.00, 3.00, 2.00, 3.00, 100.00]

default_reward = 0.00

current_temp_min = 60
current_temp_max = 70

optimal_temp_min = 55
optimal_temp_max = 59

59

In [5]:
# define specific rewards
optimal_condition_reward = 10000.0
suboptimal_condition_reward = -500.0

# function to adjust rewards based on strategy cost and temperature offset
function adjust_reward(state::Tuple{Int, Int}, action::Int)
    cost = action_costs[action]

    # Determine if the state meets the optimal temperature criteria
    if state[1] <= optimal_temp_max || state[2] >= optimal_temp_min
        base_reward = optimal_condition_reward
    else
        base_reward = suboptimal_condition_reward
    end

    # Calculate the final reward
    final_reward = base_reward - cost
    return final_reward
end

# set up rewards
rewards = Dict{Tuple{Tuple{Int, Int}, Int}, Float64}()
for state in 𝒮
    for action in 𝒜
        reward = adjust_reward(state, action)
        rewards[(state, action)] = reward
    end
end

# set up absorbing state
absorbing_state_set = Set{Tuple{Int, Int}}()
push!(absorbing_state_set, (optimal_temp_min,optimal_temp_max))

Set{Tuple{Int64, Int64}} with 1 element:
  (55, 59)

In [6]:
rewards

Dict{Tuple{Tuple{Int64, Int64}, Int64}, Float64} with 44286 entries:
  ((6, 49), 3)    => 9250.0
  ((52, 94), 3)   => 9250.0
  ((10, 81), 1)   => 10000.0
  ((29, 104), 1)  => 10000.0
  ((0, 21), 3)    => 9250.0
  ((34, 92), 3)   => 9250.0
  ((24, 39), 6)   => 3070.0
  ((13, 33), 3)   => 9250.0
  ((6, 15), 2)    => 9175.4
  ((19, 67), 5)   => 2500.0
  ((93, 118), 5)  => 2500.0
  ((58, 69), 2)   => 9175.4
  ((103, 103), 1) => 10000.0
  ((12, 99), 6)   => 3070.0
  ((31, 58), 6)   => 3070.0
  ((13, 58), 2)   => 9175.4
  ((17, 115), 5)  => 2500.0
  ((19, 62), 6)   => 3070.0
  ((38, 45), 2)   => 9175.4
  ((26, 94), 5)   => 2500.0
  ((41, 112), 6)  => 3070.0
  ((108, 114), 1) => 10000.0
  ((68, 115), 6)  => 3070.0
  ((23, 33), 5)   => 2500.0
  ((62, 108), 6)  => 3070.0
  ⋮               => ⋮

In [7]:
# reward shaping
is_reward_shaping_on = true;

if (is_reward_shaping_on == true)
    for state in 𝒮
        for action in 𝒜
           # Create a state-action pair as a coordinate
            state_action_pair = (state, action)
        
            # Check and update the reward for each state-action pair
            if !haskey(rewards, state_action_pair) && !in(state, absorbing_state_set)
                rewards[state_action_pair] = adjust_reward(state, action)
            end
        end
    end
end

In [8]:
world_model = build(MyRectangularGridWorldModel, (
        nrows=number_of_rows, ncols=number_of_cols, rewards = rewards, actions = 𝒜 ));

In [9]:
world_model

MyRectangularGridWorldModel(121, 121, Dict{Int64, Tuple{Int64, Int64}}(), Dict{Tuple{Int64, Int64}, Int64}(), Dict(5 => (-3, -3), 4 => (-2, -2), 6 => (-5, -11), 2 => (-9, -9), 3 => (-3, -3), 1 => (-1, -1)), Dict(((6, 49), 3) => 9250.0, ((52, 94), 3) => 9250.0, ((10, 81), 1) => 10000.0, ((29, 104), 1) => 10000.0, ((0, 21), 3) => 9250.0, ((34, 92), 3) => 9250.0, ((24, 39), 6) => 3070.0, ((13, 33), 3) => 9250.0, ((6, 15), 2) => 9175.4, ((19, 67), 5) => 2500.0…))

## Task 2: Q-learning Model

In [10]:
α = 0.7;  # learning rate
γ = 0.95; # discount rate
nstates = (number_of_rows*number_of_cols);
agent_model = build(MyQLearningAgentModel, (
    states = 𝒮,
    actions = 𝒜,
    α = α,
    γ = γ,
    Q = zeros(nstates,nactions)
));

## Task 3: Simulate and visualize

In [11]:
startstate = (current_temp_min,current_temp_max); # start position
number_of_episodes = 200;
number_of_iterations = 1000;

In [19]:
my_Q_dictionary = Dict{Tuple{Int,Int}, Array{Float64,2}}();
coordinate = startstate;
for i ∈ 1:number_of_episodes
    println("Running episode $i")
    # run an episode, and grab the Q
    result = simulate(agent_model, world_model, coordinate, number_of_iterations, ϵ = 0.7);
    println("Running episode $i")
    agent_model.Q = result.Q;
end
my_Q_dictionary[coordinate] = agent_model.Q;

Running episode 1


LoadError: KeyError: key (60, 70) not found

In [20]:
my_Q_dictionary

Dict{Tuple{Int64, Int64}, Matrix{Float64}}()

In [17]:
Q = my_Q_dictionary[startstate];
my_π = policy(Q);

LoadError: KeyError: key (60, 70) not found

In [14]:
Q

LoadError: UndefVarError: `Q` not defined

In [15]:
# draw the path -
p = plot();
initial_site = startstate
hit_absorbing_state = false
s = world_model.states[initial_site];
visited_sites = Set{Tuple{Int,Int}}();
push!(visited_sites, initial_site);

while (hit_absorbing_state == false)
    current_position = world_model.coordinates[s]
    a = my_π[s];
    Δ = world_model.moves[a];
    new_position =  current_position .+ Δ
    scatter!([current_position[1]],[current_position[2]], label="", showaxis=:false, msc=:black, c=:blue)
    plot!([current_position[1], new_position[1]],[current_position[2],new_position[2]], label="", arrow=true, lw=1, c=:gray)
    
    if (in(new_position, absorbing_state_set) == true || in(new_position, visited_sites) == true)
        hit_absorbing_state = true;
    elseif (haskey(world_model.states, new_position) == true)
        s = world_model.states[new_position];
        push!(visited_sites, new_position);
    else
        hit_absorbing_state = true; # we drove off the map
    end
end

# draw the grid -
for s ∈ 𝒮
    current_position = world_model.coordinates[s]
    a = my_π[s];
    Δ = world_model.moves[a];
    new_position =  current_position .+ Δ
    
    if (haskey(rewards, current_position) == true && rewards[current_position] == charging_reward)
        scatter!([current_position[1]],[current_position[2]], label="", showaxis=:false, c=:green, ms=4)
    elseif (haskey(rewards, current_position) == true && rewards[current_position] == lava_reward)
        scatter!([current_position[1]],[current_position[2]], label="", showaxis=:false, c=:red, ms=4)
    elseif (in(current_position, soft_wall_set) == true)
        scatter!([current_position[1]],[current_position[2]], label="", showaxis=:false, c=:gray69, ms=4)
    else
        if (is_reward_shaping_on == true)
            new_color = weighted_color_mean(rbf(current_position, (5,6), σ = σ), colorant"green", colorant"white")
            scatter!([current_position[1]],[current_position[2]], label="", showaxis=:false, msc=:lightgray, c=new_color)
        else
            scatter!([current_position[1]],[current_position[2]], label="", showaxis=:false, msc=:black, c=:white)
        end
    end
end
current()

LoadError: KeyError: key (60, 70) not found